In [69]:
import pandas as pd
import re

In [90]:
import pandas as pd
import re

class Prompting:
    def __init__(self):
        excel_file_path = 'prompts.xlsx'
        self.prompts = pd.read_excel(excel_file_path)
        self.chat = chatGPT()
        
        self.occupation = "Lawyer"
        self.gov_tasks = self.getGovTask(self.occupation)
        self.chatGPTPrompting(self.gov_tasks)
        
    def getGovTask(self, occupation):
        df = pd.read_csv("Task Statements.csv")
        occ_tasks = df.loc[df.Title == occupation + "s"]
        gov_tasks = list(occ_tasks.Task)
        return gov_tasks
        
    def splitResponse(self, response):
        return re.sub(r'\d+\.\s', '', response).split("\n")

    def getPrompt(self, i, *vars):
        prompt = self.prompts.Prompt[i - 1]
        prompt = re.sub(r'\{.*?\}', '{}', prompt)
        prompt = prompt.format(*vars)
        return prompt

    def chatGPTPrompting(self, occupation,tasks):
        with open("../output/occupation.txt", "a") as file:
            for task in tasks[:1]:
                prompt1 = self.getPrompt(1, task) 
                response1 = self.chat.ask(prompt1)
                file.write(f"\n\n\nPROMPT1 {prompt1}\n{response1}\n")

                prompt2 = self.getPrompt(2, response1.replace("\n", "")) 
                response2 = self.chat.ask(prompt2)
                file.write(f"\n\n\nPROMPT2 {prompt2}\n{response2}\n")

                subtasks = self.splitResponse(response2)
    
                for subtask in subtasks[:1]:
                    prompt3 = self.getPrompt(3, occupation, subtask) 
                    response3 = self.chat.ask(prompt3)
                    file.write(f"\n\n\nPROMPT3 {prompt3}\n{response3}\n")

                    prompt4 = self.getPrompt(4, response3) 
                    response4 = self.chat.ask(prompt4)
                    file.write(f"\n\n\nPROMPT4 {prompt4}\n{response4}\n")





x = Prompting()



 PROMPT1 Generate subtasks for: Analyze the probable outcomes of cases, using knowledge of legal precedents.. Respond in concise 100 words in numbered list of max 3.
1. Case Research: Review and analyze all case information, determining the facts and context. Understand the specific laws applied to the case and how they have been interpreted in previous similar cases. Look for legal precedents that could potentially influence the outcome of the current case. 

2. Legal Precedent Application: Apply knowledge of former verdicts, highlighting similarities and differences between present case and past cases. Evaluate how these precedents could shape the verdict of the current case, providing justification for how these could support your arguments.

3. Project Potential Outcomes: Based on the application of precedents, predict potential outcomes of the case. Cover both best-case and worst-case scenarios, and prepare strategies for each potential verdict – acquittal, conviction, dismissal,

In [89]:
from dotenv import load_dotenv
from os import getenv
from openai import OpenAI

# Load environment variables from .env file
load_dotenv("../.env")

class chatGPT():
    def __init__(self):
        # Get API key from environment variable
        my_key = getenv("OPENAI_API_KEY")
        if my_key is None:
            raise ValueError("API key not found in .env file")
        
        # Initialize OpenAI client with API key
        self.client = OpenAI(api_key=my_key)

    def ask(self, q):
        stream = self.client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": q}],
            stream=True,
        )
        response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                response += chunk.choices[0].delta.content

        self.response = response
        return response

# Example usage
chatbot = chatGPT()
response = chatbot.ask("Hello, how are you?")
print(response)


Hello! As an artificial intelligence, I don't have feelings, but I'm here and ready to help you. How can I assist you today?
